# APRIORI

## ALGORITMO DE ASSOCIAÇÃO

# LIB's

In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import datetime

from apyori import apriori

# DATASET

In [7]:
df = pd.read_csv('data.csv', sep = ';', decimal = ',')
df.head()

,data_pedido,hora_pedido,tipo_entrega,tempo_minutos,borda,refrigerante,Sobremesa,valor_pizza,valor_total
0,29/10/2011,21:00:00,Entrega,55,N,N,S,38.5,38.5
1,29/10/2011,21:10:00,Entrega,45,N,N,S,38.5,38.5
2,30/10/2011,20:17:00,Entrega,51,N,N,S,38.5,38.5
3,01/11/2011,21:10:00,Entrega,50,N,S,S,34.0,38.5
4,04/11/2011,20:37:00,Entrega,33,N,S,S,34.0,38.5


In [9]:
df.shape

(1000, 9)

In [8]:
df.dtypes

data_pedido       object
hora_pedido       object
tipo_entrega      object
tempo_minutos      int64
borda             object
refrigerante      object
Sobremesa         object
valor_pizza      float64
valor_total      float64
dtype: object